In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import string
import re
from nltk.stem import PorterStemmer

In [2]:
df=pd.read_csv('spam_ham_dataset.csv')

In [3]:
text=df['text'][0]
print(text)

Subject: enron methanol ; meter # : 988291
this is a follow up to the note i gave you on monday , 4 / 3 / 00 { preliminary
flow data provided by daren } .
please override pop ' s daily volume { presently zero } to reflect daily
activity you can obtain from gas control .
this change is needed asap for economics purposes .


In [4]:
st=PorterStemmer()
def clean_text(text):
    #lower case whole text
    text=text.lower()
    #removing digits
    text=re.sub(r'\d+','',text)
    #remove punctuations
    text=''.join([i for i in text if i not in string.punctuation])
    #removes extra space from the beginning or ending of sent
    text=text.strip()
    text=text.replace('subject','').strip()
    text=' '.join([st.stem(word) for word in text.split()])
    return text

In [5]:
df['clean_text']=df['text'].apply(clean_text)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer(stop_words='english')

In [7]:
#Exammple
#Term Frequency - Inverse Document Frequency
tfidf=vectorizer.fit_transform(['i like apple','i like oranges', 'i like apple and oranges'])

pd.DataFrame(tfidf.toarray(), columns=vectorizer.get_feature_names_out())

,apple,like,oranges
0,0.789807,0.613356,0.000000
1,0.000000,0.613356,0.789807
2,0.619805,0.481334,0.619805


In [8]:
X=vectorizer.fit_transform(df['clean_text'])
y=df['label_num']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

from sklearn.naive_bayes import MultinomialNB
nb=MultinomialNB()

nb.fit(X_train,y_train)
y_pred=nb.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      1.00      0.96       739
           1       1.00      0.78      0.87       296

    accuracy                           0.94      1035
   macro avg       0.96      0.89      0.91      1035
weighted avg       0.94      0.94      0.93      1035



In [9]:
def predict_email(text):
    global pred
    cleaned_text=clean_text(text)
    vectorized=vectorizer.transform([cleaned_text])
    pred=nb.predict_proba(vectorized)[0]
    if pred[1]>0.3:
        print('This seems like a Spam email')
    else:
        print('This seems like a Genuine email')

In [14]:
exmaple_text='This is to inform you that you have been promoted.'
predict_email(exmaple_text)

This seems like a Genuine email
